# ChatterBot 範例程式碼

## 載入套件
若希望能看到 ChatBot 的決策過程，可以把最後兩行取消註解

In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

# Uncomment the following line to enable verbose logging
# import logging
# logging.basicConfig(level=logging.INFO)

## 基本用法

ChatterBot 最重要的部分有二：
- ChatBot
- Trainer

先創造出 ChatBot 並為其命名，再根據訓練資料型態指定 trainer ，即可進行訓練。訓練完成後使用指令`.get_response()`可以和 ChatBot 對話，而對話的同時也是在訓練 ChatBot，擴增它的資料庫。

In [2]:
chatbot = ChatBot('YOUR CHATBOT NAME')

# Create a new trainer for the chatbot
trainer = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the english corpus
trainer.train("chatterbot.corpus.english")

# Get a response to an input statement
chatbot.get_response("Good morning! How are you doing?")

Training ai.yml: [##                  ] 8%

C:\Users\user\Anaconda3\lib\site-packages\chatterbot\corpus.py:59: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(data_file)


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


<Statement text:I am doing well, how about you?>

## 使用list格式的資料訓練chatbot
若使用的語料不是以 chatterbot-corpus 那樣的格式儲存，也可以將資料整理成 list，使用`ListTrainer`來訓練。

In [3]:
from chatterbot.trainers import ListTrainer

chatbot = ChatBot('Example Bot')

# Start by training our bot with the ChatterBot corpus data
trainer = ListTrainer(chatbot)

trainer.train([
    'Hello, how are you?',
    'I am doing well.',
    'That is good to hear.',
    'Thank you'
])

# You can train with a second list of data to add response variations

trainer.train([
    'Hello, how are you?',
    'I am great.',
    'That is awesome.',
    'Thanks'
])

# Now let's get a response to a greeting
response = chatbot.get_response('Hello, how are you?')
print(response)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
I am doing well.


# 邏輯適配器
在 ChatterBot 中，你可以選擇 ChatBot 在尋找適當回應的邏輯。`logic_adapter`的預設值為 ['chatterbot.logic.BestMatch']，即找出資料庫中與輸入相近的句子，再取出這些句子的回應句，根據使用者說出這些回應句的頻率，來決定要以哪個句子作為最終回應。　　　　

除了BestMatch之外，套件中也有其他現成的邏輯適配器可以使用。若同時使用了多個邏輯適配器，則根據每個適配器給出的信心水準，採用信心水準最高的適配器生成的回應。

## 單位換算邏輯適配器
UnitConversion 可以回答關於單位換算的問題，例如一公里等於多少公尺等問題。

In [4]:
bot = ChatBot('Unit Converter',
              logic_adapters=['chatterbot.logic.UnitConversion',]
             )

questions = [
    'How many meters are in a kilometer?',
    'How many meters are in one inch?',
    '0 celsius to fahrenheit',
    'one hour is how many minutes ?'
]

# Prints the convertion given the specific question
for question in questions:
    response = bot.get_response(question)
    print(question + ' -  Response: ' + response.text)

How many meters are in a kilometer? -  Response: 1000.0
How many meters are in one inch? -  Response: 0.0254
0 celsius to fahrenheit -  Response: 31.999999999999936
one hour is how many minutes ? -  Response: 60.0


## 最佳匹配邏輯適配器(預設回應)
可以為適配器設定預設的回應，當其信心水準低於某一閥值時，令其回傳預設的回應，例如「我不明白」等，可以避免出現不恰當的回應。

In [5]:
# Create a new instance of a ChatBot
bot = ChatBot(
    'Example Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'I am sorry, but I do not understand.',
            'maximum_similarity_threshold': 0.90
        }
    ]
)

trainer = ListTrainer(bot)

# Train the chat bot with a few responses
trainer.train([
    'How can I help you?',
    'I want to create a chat bot',
    'Have you read the documentation?',
    'No, I have not',
    'This should help get you started: http://chatterbot.rtfd.org/en/latest/quickstart.html'
])

# Get a response for some unexpected input
response = bot.get_response('How do I make an omelette?')
print(response)

List Trainer: [####################] 100%
Sort of.


## 數學及時間邏輯適配器
這兩個適配器分別適用於回應數學問題及時間問題。

In [6]:
bot = ChatBot(
    'Math & Time Bot',
    logic_adapters=[
        'chatterbot.logic.MathematicalEvaluation',
        'chatterbot.logic.TimeLogicAdapter'
    ]
)

# Print an example of getting one math based response
response = bot.get_response('What is 4 + 9?')
print(response)

# Print an example of getting one time based response
response = bot.get_response('What time is it?')
print(response)

4 + 9 = 13
The current time is 07:53 AM


## 特定回答邏輯適配器
使用 SpecificResponseAdapter 可以對指定句子產生固定回答。(這裡原始碼有點小bug，如果輸出結果不是 'Ok, here is a link: http://chatterbot.rtfd.org' 也沒關係，可以參考https://github.com/gunthercox/ChatterBot/issues/1677 這個 Issue。

In [7]:
# Create a new instance of a ChatBot
bot = ChatBot(
    'Exact Response Example Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch'
        },
        {
            'import_path': 'chatterbot.logic.SpecificResponseAdapter',
            'input_text': 'Help me!',
            'output_text': 'Ok, here is a link: http://chatterbot.rtfd.org'
        }
    ]
)
# Get a response given the specific input
response = bot.get_response('Help me!')
print(response)

Right this way.


# 其他範例
## 從回饋學習
示範如何讓 ChatBot 根據使用者對於回答所給予的評價學習，避免 ChatBot 不適當的回答又被加入資料庫中。

In [8]:
from chatterbot.conversation import Statement

"""
This example shows how to create a chat bot that
will learn responses based on an additional feedback
element from the user.
"""

# Create a new instance of a ChatBot
bot = ChatBot(
    'Feedback Learning Bot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter'
)


def get_feedback():

    text = input()

    if 'yes' in text.lower():
        return True
    elif 'no' in text.lower():
        return False
    else:
        print('Please type either "Yes" or "No"')
        return get_feedback()


print('Type something to begin...')

# The following loop will execute each time the user enters input
while True:
    try:
        input_statement = Statement(text=input())
        response = bot.generate_response(
            input_statement
        )

        print('\n Is "{}" a coherent response to "{}"? \n'.format(
            response.text,
            input_statement.text
        ))
        if get_feedback() is False:
            print('please input the correct one')
            correct_response = Statement(text=input())
            bot.learn_response(correct_response, input_statement)
            print('Responses added to bot!')

    # Press ctrl-c or ctrl-d on the keyboard to exit
    except (KeyboardInterrupt, EOFError, SystemExit):
        break

Type something to begin...
A


No value for search_text was available on the provided input



 Is "Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think." a coherent response to "A"? 

test
Please type either "Yes" or "No"
NO
please input the correct one
No
Responses added to bot!


## export
有時候可能會需要儲存 ChatBot 累積的訓練資料，以便將來訓練新的 ChatBot 使用。指令`.export_for_training()`能夠完成這件事。

In [9]:
'''
This is an example showing how to create an export file from
an existing chat bot that can then be used to train other bots.
'''

chatbot = ChatBot('Export Example Bot')

# First, lets train our bot with some data
trainer = ChatterBotCorpusTrainer(chatbot)

trainer.train('chatterbot.corpus.english')

# Now we can export the data to a file
trainer.export_for_training('./my_export.json')

Training ai.yml: [###                 ] 13%

C:\Users\user\Anaconda3\lib\site-packages\chatterbot\corpus.py:59: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(data_file)


Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


### 補充
- ChatterBot 會自動儲存上個 ChatBot 接收過的訓練資料，若不希望新的 ChatBot 受到干擾，可以在建立 ChatBot之後加上 `chatbot.storage.drop()`，或手動刪除資料夾中的 db.sqlite3 等檔案。
- 個人認為 ChatterBot 並不算是功能很豐富、完整的套件，但是它能接受的客製化程度很高，像是 logic_adapter、trainer、preprocessor 等等，幾乎所有部分都是可以自己依需求寫好再接上去的(不過這樣做也挺麻煩就是了)。